# Exploratory analysis of GSC metabolomics data
Data collected and provided by Olga

### Cleanup data

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from collections import Counter

In [3]:
# Import the metabolomics data
metabolomics_df = pd.read_csv('C:/research/raw_data/20200207_SU2C_All_Knowns.csv', index_col=0)
metabolomics_df.head()

,DateProcessed,TotalRNA,Trial,Lab,CellLine,SampleType,Oxygen,Replicate,Adenosine_OR_Deoxyguanosine,alpha_Ketoglutarate,...,UMP,Uracil,Uridine,X2_Aminoadipic_Acid,X2HG,X3_Methyl_2_Oxopentoate,X4_Methyl_2_Oxopentoate,X5_Hydroxy_Trp,Xanthine,Xylulose_5P
FileNumber,,,,,,,,,,,,,,,,,,,,,
1,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,1.0,0.648389,0.233167,...,0.031392,0.083978,1.407985,23.773639,0.055788,27823.74426,18542.49087,0.001257,4.165198,7.158272
2,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,2.0,0.607384,0.160450,...,0.033101,0.090519,1.306536,27.819868,0.060646,26961.89141,25807.97566,0.001185,4.298604,7.028919
3,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,3.0,0.677430,0.161100,...,0.039041,0.130953,1.308885,28.465713,0.068040,24184.86136,19483.41231,0.001135,4.000341,8.386796
4,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,4.0,0.658178,0.147507,...,0.032998,0.082612,1.355951,28.223934,0.041304,25654.80883,21779.36667,0.001220,4.327418,6.882724
5,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,5.0,0.552922,0.179677,...,0.037922,0.074435,1.106163,23.120153,0.055203,28370.42718,21526.05841,0.001093,3.884993,8.677013


In [ ]:
# Correct mistakes in the original dataframe
# ('X20160316', np.int64(1), 'G571', 'Normoxia') was a mistake, the date for this should be 'X20160406'
metabolomics_df.loc[(metabolomics_df['DateProcessed'] == 'X20160316') &
                    (metabolomics_df['Trial'] == 1) &
                    (metabolomics_df['CellLine'] == 'G571') &
                    (metabolomics_df['Oxygen'] == 'Normoxia'), 'DateProcessed'] = 'X20160406'
                
# ('X20161124', np.int64(1), 'BT301', 'Normoxia') only has 22 rows because there are only 5 replicates for both "Cells" and "Secreted" metabolites

# ('X20161018', np.int64(2), 'G620', 'Normoxia') only has 21 rows because there are only 4 replicates for "Cells" and only 5 replicates for "MediaControl"

# ('X20161018', np.int64(2), 'G564', 'Normoxia') only has 11 rows because there are only 5 replicates for "Cells" and only 6 replicates for "Secreted", no controls

# ('X20160316', np.int64(1), 'G729', 'Normoxia') has 34 rows because there are 12 replicates each for "Cells" and "Secreted"

# There are 6 dataframes with 14 rows each. These data are from the Weiss lab and their cell lines begin with "BT"
# ('X20170203', np.int64(1), 'BT147', 'Normoxia')
# ('X20170330', np.int64(1), 'BT169', 'Normoxia')
# ('X20170330', np.int64(1), 'BT189', 'Normoxia')
# ('X20170330', np.int64(1), 'BT301', 'Normoxia')
# ('X20170330', np.int64(1), 'BT69', 'Normoxia')
# ('X20170330', np.int64(1), 'BT94', 'Normoxia')

# There are 7 dataframes with 15 rows each. These data are from the Weiss lab and their cell lines begin with "BT"
# ('X20170203', np.int64(1), 'BT48', 'Normoxia')
# ('X20170203', np.int64(1), 'BT50', 'Normoxia')
# ('X20170203', np.int64(1), 'BT73', 'Normoxia')
# ('X20170203', np.int64(1), 'BT75', 'Normoxia')
# ('X20170203', np.int64(1), 'BT89', 'Normoxia')
# ('X20170330', np.int64(1), 'BT248', 'Normoxia')
# ('X20170525', np.int64(1), 'BT238', 'Normoxia')

In [115]:
# How many unique cell lines are here? 68
metabolomics_df['CellLine'].nunique()

34

In [123]:
# For simplicity, only look at data of cell lines from the Dirks lab for now

# Ensure the cell line column is string type
metabolomics_df['CellLine'] = metabolomics_df['CellLine'].astype('string')

# Remove all cell lines starting with "HF" in the full dataframe, these are human fetal
metabolomics_df = metabolomics_df[~metabolomics_df['CellLine'].str.startswith('HF')]

# Keep only rows where the Lab column is "Dirks"
metabolomics_df = metabolomics_df[metabolomics_df['Lab'] == 'Dirks']

# Remove rows where SampleType is "Cells" and TotalRNA is NaN
# These samples cannot be normalized to RNA amount
metabolomics_df = metabolomics_df[~((metabolomics_df['SampleType'] == 'Cells') & (metabolomics_df['TotalRNA'].isna()))]

# How many unique cell lines are here? Now 34
metabolomics_df['CellLine'].nunique()

34

In [112]:
# Subset the dataframe for a specific cell line, e.g., 'G523'
G523_df = metabolomics_df[metabolomics_df['CellLine'] == 'G523']
G523_df

,DateProcessed,TotalRNA,Trial,Lab,CellLine,SampleType,Oxygen,Replicate,Adenosine_OR_Deoxyguanosine,alpha_Ketoglutarate,...,UMP,Uracil,Uridine,X2_Aminoadipic_Acid,X2HG,X3_Methyl_2_Oxopentoate,X4_Methyl_2_Oxopentoate,X5_Hydroxy_Trp,Xanthine,Xylulose_5P
FileNumber,,,,,,,,,,,,,,,,,,,,,
163,X20160621,NaN,1,Dirks,G523,Coating,Normoxia,1.0,0.267561,0.031166,...,0.037939,0.472435,0.230827,0.704831,0.239954,7.312621,9.261954,0.000102,0.061448,4.121885
164,X20160621,NaN,1,Dirks,G523,Coating,Normoxia,2.0,0.389552,0.043990,...,0.030166,0.194152,0.100240,1.213483,0.370319,11.247858,9.712298,0.000094,0.029010,4.096100
165,X20160621,NaN,1,Dirks,G523,Coating,Normoxia,3.0,0.261106,0.033545,...,0.023404,0.226679,0.092935,0.699387,0.223355,9.859366,8.602156,0.000090,0.029153,4.196879
166,X20160621,NaN,1,Dirks,G523,Coating,Normoxia,4.0,0.211518,0.029799,...,0.029781,0.208066,0.096608,0.239352,0.261994,9.185934,5.689928,0.000105,0.022313,4.170834
167,X20160621,NaN,1,Dirks,G523,Coating,Normoxia,5.0,0.149310,0.042557,...,0.024108,0.219265,0.111523,0.280174,0.261121,9.785174,8.193086,0.000119,0.022712,4.399674
168,X20160621,NaN,1,Dirks,G523,Coating,Normoxia,6.0,0.126174,0.034243,...,0.024884,0.160767,0.077155,0.283596,0.170843,7.672585,8.656908,0.000098,0.020306,3.888903
169,X20160621,NaN,1,Dirks,G523,MediaControl,Normoxia,1.0,0.294595,0.067076,...,0.028304,0.036006,0.097526,0.395041,0.403458,12.822295,10.281107,0.000657,0.106439,7.387533
170,X20160621,NaN,1,Dirks,G523,MediaControl,Normoxia,2.0,0.282187,0.048903,...,0.030559,0.126570,0.361052,0.911564,0.407024,17.448724,36.773607,0.000747,0.214067,8.240283
171,X20160621,NaN,1,Dirks,G523,MediaControl,Normoxia,3.0,0.206038,0.115745,...,0.028734,0.054968,0.144775,0.459707,0.432076,21.208568,35.470553,0.000728,0.144201,8.544129


In [64]:
# Check the datatypes of the columns in the original dataframe
print(metabolomics_df.dtypes)

DateProcessed               object
TotalRNA                   float64
Trial                        int64
Lab                         object
CellLine                    object
                            ...   
X3_Methyl_2_Oxopentoate    float64
X4_Methyl_2_Oxopentoate    float64
X5_Hydroxy_Trp             float64
Xanthine                   float64
Xylulose_5P                float64
Length: 82, dtype: object


In [124]:
# Split the full dataframe into small dataframes based on unique combinations of DateProcessed, Trial, CellLine, and Oxygen
import numpy as np

# Split the G523 large dataframe into two dataframes
# Create a dictionary of dataframes
dfs_dict = {key: group for key, group in metabolomics_df.groupby(['DateProcessed', 'Trial', 'Lab', 'CellLine', 'Oxygen'])}
dfs_dict

print(list(dfs_dict.keys()))
print("The main df has been split into " + str(len(dfs_dict)) + " dataframes.")

# Example: access the dataframe for a specific combination
# subset_df = dfs_dict[('X20180717', np.int64(1), 'G809', 'Normoxia')]
# print(subset_df)

[('X20160316', np.int64(1), 'Dirks', 'G729', 'Normoxia'), ('X20160406', np.int64(1), 'Dirks', 'G571', 'Normoxia'), ('X20160502', np.int64(1), 'Dirks', 'G549', 'Normoxia'), ('X20160502', np.int64(1), 'Dirks', 'G566', 'Normoxia'), ('X20160502', np.int64(1), 'Dirks', 'G583', 'Normoxia'), ('X20160502', np.int64(1), 'Dirks', 'G620', 'Normoxia'), ('X20160621', np.int64(1), 'Dirks', 'G523', 'Normoxia'), ('X20160621', np.int64(1), 'Dirks', 'G564', 'Normoxia'), ('X20160916', np.int64(2), 'Dirks', 'G583', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G523', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G549', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G564', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G620', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G729', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'BT301', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'BT50', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'BT94', 'Normoxia'), ('X20161124', np.int64(1), 'D

In [125]:
# Iterate through the dictionary and print the number of rows in each dataframe
row_counts = {key: df.shape[0] for key, df in dfs_dict.items()}
print(row_counts)

# Count the frequency of each row count
count_freq = Counter(row_counts.values())
print(count_freq)

{('X20160316', np.int64(1), 'Dirks', 'G729', 'Normoxia'): 34, ('X20160406', np.int64(1), 'Dirks', 'G571', 'Normoxia'): 18, ('X20160502', np.int64(1), 'Dirks', 'G549', 'Normoxia'): 23, ('X20160502', np.int64(1), 'Dirks', 'G566', 'Normoxia'): 24, ('X20160502', np.int64(1), 'Dirks', 'G583', 'Normoxia'): 24, ('X20160502', np.int64(1), 'Dirks', 'G620', 'Normoxia'): 24, ('X20160621', np.int64(1), 'Dirks', 'G523', 'Normoxia'): 24, ('X20160621', np.int64(1), 'Dirks', 'G564', 'Normoxia'): 23, ('X20160916', np.int64(2), 'Dirks', 'G583', 'Normoxia'): 24, ('X20161018', np.int64(2), 'Dirks', 'G523', 'Normoxia'): 24, ('X20161018', np.int64(2), 'Dirks', 'G549', 'Normoxia'): 23, ('X20161018', np.int64(2), 'Dirks', 'G564', 'Normoxia'): 11, ('X20161018', np.int64(2), 'Dirks', 'G620', 'Normoxia'): 21, ('X20161018', np.int64(2), 'Dirks', 'G729', 'Normoxia'): 24, ('X20161124', np.int64(1), 'Dirks', 'BT301', 'Normoxia'): 22, ('X20161124', np.int64(1), 'Dirks', 'BT50', 'Normoxia'): 24, ('X20161124', np.int64

In [126]:
# Given the number of rows, print the keys (i.e., the combinations) that have that number of rows
def print_keys_for_row_count(row_count):
    keys = [key for key, count in row_counts.items() if count == row_count]
    for key in keys:
        print(key)

print_keys_for_row_count(12)

In [128]:
# Pull specific dataframe from the dictionary 
dfs_dict[('X20160316', np.int64(1), 'Dirks', 'G729', 'Normoxia')]

,DateProcessed,TotalRNA,Trial,Lab,CellLine,SampleType,Oxygen,Replicate,Adenosine_OR_Deoxyguanosine,alpha_Ketoglutarate,...,UMP,Uracil,Uridine,X2_Aminoadipic_Acid,X2HG,X3_Methyl_2_Oxopentoate,X4_Methyl_2_Oxopentoate,X5_Hydroxy_Trp,Xanthine,Xylulose_5P
FileNumber,,,,,,,,,,,,,,,,,,,,,
1,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,1.0,0.648389,0.233167,...,0.031392,0.083978,1.407985,23.773639,0.055788,27823.744260,18542.490870,0.001257,4.165198,7.158272
2,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,2.0,0.607384,0.160450,...,0.033101,0.090519,1.306536,27.819868,0.060646,26961.891410,25807.975660,0.001185,4.298604,7.028919
3,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,3.0,0.677430,0.161100,...,0.039041,0.130953,1.308885,28.465713,0.068040,24184.861360,19483.412310,0.001135,4.000341,8.386796
4,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,4.0,0.658178,0.147507,...,0.032998,0.082612,1.355951,28.223934,0.041304,25654.808830,21779.366670,0.001220,4.327418,6.882724
5,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,5.0,0.552922,0.179677,...,0.037922,0.074435,1.106163,23.120153,0.055203,28370.427180,21526.058410,0.001093,3.884993,8.677013
6,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,6.0,0.571963,0.165721,...,0.041156,0.072914,1.164580,23.703608,0.070233,24100.436500,26821.829730,0.001116,3.669924,9.284897
7,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,7.0,0.796806,0.232587,...,0.037406,0.064332,1.128247,16.168259,0.119402,21949.179960,18623.542820,0.001098,3.184933,9.451866
8,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,8.0,0.791972,0.173989,...,0.039668,0.076015,0.980611,15.931988,0.064417,21194.106520,20341.938660,0.000993,2.845267,10.067808
9,X20160316,NaN,1,Dirks,G729,Secreted,Normoxia,9.0,0.721547,0.102578,...,0.037375,0.071462,1.068608,18.777159,0.040971,20260.540500,19921.617490,0.001088,3.388513,9.563010


In [129]:
keys_with_24_rows = [key for key, count in row_counts.items() if count == 24]
print("There are " + str(len(keys_with_24_rows)) + " dataframes with 24 rows each:")
print(keys_with_24_rows)


There are 36 dataframes with 24 rows each:
[('X20160502', np.int64(1), 'Dirks', 'G566', 'Normoxia'), ('X20160502', np.int64(1), 'Dirks', 'G583', 'Normoxia'), ('X20160502', np.int64(1), 'Dirks', 'G620', 'Normoxia'), ('X20160621', np.int64(1), 'Dirks', 'G523', 'Normoxia'), ('X20160916', np.int64(2), 'Dirks', 'G583', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G523', 'Normoxia'), ('X20161018', np.int64(2), 'Dirks', 'G729', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'BT50', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'BT94', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'G477', 'Normoxia'), ('X20161124', np.int64(1), 'Dirks', 'G626', 'Normoxia'), ('X20170329', np.int64(1), 'Dirks', 'G607', 'Normoxia'), ('X20170329', np.int64(1), 'Dirks', 'G648', 'Normoxia'), ('X20170329', np.int64(1), 'Dirks', 'G691', 'Normoxia'), ('X20170329', np.int64(1), 'Dirks', 'G752', 'Normoxia'), ('X20170329', np.int64(2), 'Dirks', 'G683', 'Normoxia'), ('X20170710', np.int64(1), 'Dirks', 'G637', 

In [130]:
dfs_dict[keys_with_24_rows[0]]

,DateProcessed,TotalRNA,Trial,Lab,CellLine,SampleType,Oxygen,Replicate,Adenosine_OR_Deoxyguanosine,alpha_Ketoglutarate,...,UMP,Uracil,Uridine,X2_Aminoadipic_Acid,X2HG,X3_Methyl_2_Oxopentoate,X4_Methyl_2_Oxopentoate,X5_Hydroxy_Trp,Xanthine,Xylulose_5P
FileNumber,,,,,,,,,,,,,,,,,,,,,
115,X20160502,10.966134,1,Dirks,G566,Cells,Normoxia,1.0,1.299610,0.565064,...,0.600054,0.069079,0.075891,56.280940,1.004178,373.371199,467.232569,0.001468,0.031955,30.855174
116,X20160502,12.726160,1,Dirks,G566,Cells,Normoxia,2.0,1.182842,0.541490,...,0.452679,0.053064,0.079220,64.412472,0.974184,280.905792,444.395552,0.001125,0.028618,32.198169
117,X20160502,14.149547,1,Dirks,G566,Cells,Normoxia,3.0,1.218267,0.543315,...,0.414936,0.061730,0.053737,37.654088,0.878812,390.332275,432.870207,0.001118,0.024614,29.792256
118,X20160502,19.906782,1,Dirks,G566,Cells,Normoxia,4.0,1.020001,0.232111,...,0.271748,0.076974,0.072578,28.354648,0.500097,184.330164,232.022290,0.000748,0.023723,21.164382
119,X20160502,38.477532,1,Dirks,G566,Cells,Normoxia,5.0,0.984195,0.147327,...,0.189617,0.064827,0.057501,17.817535,0.293561,85.594192,113.026060,0.000404,0.018531,13.892269
120,X20160502,20.165580,1,Dirks,G566,Cells,Normoxia,6.0,0.965782,0.391582,...,0.271167,0.246682,0.128019,31.695634,0.532339,193.069946,217.688135,0.000904,0.064488,20.446712
121,X20160502,NaN,1,Dirks,G566,Coating,Normoxia,1.0,0.128267,0.074137,...,0.497142,0.082865,0.012898,0.378615,0.058269,18.311374,12.717690,0.000321,0.030378,3.113568
122,X20160502,NaN,1,Dirks,G566,Coating,Normoxia,2.0,0.107052,0.101026,...,0.593290,0.075124,0.010348,0.338548,0.074797,8.251821,7.125509,0.000430,0.037656,3.345292
123,X20160502,NaN,1,Dirks,G566,Coating,Normoxia,3.0,0.114081,0.103635,...,0.655028,0.057127,0.010047,0.533288,0.081104,8.399938,5.524482,0.000362,0.029058,3.216459


In [ ]:
# Split the dataframe for keys_with_24_rows[0] by SampleType
sampletype_dfs = {sample_type: group for sample_type, group in dfs_dict[keys_with_24_rows[0]].groupby('SampleType')}
sampletype_dfs

{'Cells':            DateProcessed   TotalRNA  Trial    Lab CellLine SampleType  \
 FileNumber                                                              
 115            X20160502  10.966134      1  Dirks     G566      Cells   
 116            X20160502  12.726160      1  Dirks     G566      Cells   
 117            X20160502  14.149547      1  Dirks     G566      Cells   
 118            X20160502  19.906782      1  Dirks     G566      Cells   
 119            X20160502  38.477532      1  Dirks     G566      Cells   
 120            X20160502  20.165580      1  Dirks     G566      Cells   
 
               Oxygen  Replicate  Adenosine_OR_Deoxyguanosine  \
 FileNumber                                                     
 115         Normoxia        1.0                     1.299610   
 116         Normoxia        2.0                     1.182842   
 117         Normoxia        3.0                     1.218267   
 118         Normoxia        4.0                     1.020001   
 119   

In [ ]:
# Normalize the 'Cells' dataframe to TotalRNA
start_col = 'Adenosine_OR_Deoxyguanosine'
cols_to_normalize = sampletype_dfs['Cells'].columns[
    sampletype_dfs['Cells'].columns.get_loc(start_col):
]
normalized_cells_df = sampletype_dfs['Cells'][cols_to_normalize].div(sampletype_dfs['Cells']['TotalRNA'], axis=0)
print(normalized_cells_df)

           DateProcessed   TotalRNA  Trial    Lab CellLine SampleType  \
FileNumber                                                              
115            X20160502  10.966134      1  Dirks     G566      Cells   
116            X20160502  12.726160      1  Dirks     G566      Cells   
117            X20160502  14.149547      1  Dirks     G566      Cells   
118            X20160502  19.906782      1  Dirks     G566      Cells   
119            X20160502  38.477532      1  Dirks     G566      Cells   
120            X20160502  20.165580      1  Dirks     G566      Cells   

              Oxygen  Replicate  Adenosine_OR_Deoxyguanosine  \
FileNumber                                                     
115         Normoxia        1.0                     1.299610   
116         Normoxia        2.0                     1.182842   
117         Normoxia        3.0                     1.218267   
118         Normoxia        4.0                     1.020001   
119         Normoxia        5.0

In [140]:
# Get the subseted dataframe containing only the 'Coating' samples and get the median of each metabolite column
# Get all columns starting from 'Adenosine_OR_Deoxyguanosine' onwards
coating_cols = sampletype_dfs['Coating'].columns[
    sampletype_dfs['Coating'].columns.get_loc(start_col):
]

# Calculate the average for each of these columns
coating_median = sampletype_dfs['Coating'][coating_cols].median()
print(coating_median)


Adenosine_OR_Deoxyguanosine    0.108711
alpha_Ketoglutarate            0.086429
AMP_OR_dGMP                    0.048180
Argininosuccinate              0.038011
beta_NAD                       0.012438
                                 ...   
X3_Methyl_2_Oxopentoate        8.053651
X4_Methyl_2_Oxopentoate        5.461656
X5_Hydroxy_Trp                 0.000333
Xanthine                       0.028056
Xylulose_5P                    3.091143
Length: 74, dtype: float64


In [153]:
# Subtract the coating_median from each corresponding column in sampletype_dfs['Cells']
cells_df = sampletype_dfs['Cells']
metabolite_cols = coating_median.index
coating_subtracted = cells_df.copy()
coating_subtracted[metabolite_cols] = cells_df[metabolite_cols] - coating_median
print(coating_subtracted)

           DateProcessed   TotalRNA  Trial    Lab CellLine SampleType  \
FileNumber                                                              
115            X20160502  10.966134      1  Dirks     G566      Cells   
116            X20160502  12.726160      1  Dirks     G566      Cells   
117            X20160502  14.149547      1  Dirks     G566      Cells   
118            X20160502  19.906782      1  Dirks     G566      Cells   
119            X20160502  38.477532      1  Dirks     G566      Cells   
120            X20160502  20.165580      1  Dirks     G566      Cells   

              Oxygen  Replicate  Adenosine_OR_Deoxyguanosine  \
FileNumber                                                     
115         Normoxia        1.0                     1.190899   
116         Normoxia        2.0                     1.074131   
117         Normoxia        3.0                     1.109556   
118         Normoxia        4.0                     0.911291   
119         Normoxia        5.0

In [157]:
# Calculate the half-min values for each metabolite column
half_min_values = coating_subtracted[metabolite_cols].apply(lambda col: col[col > 0].min() / 2 if (col > 0).any() else np.nan)
print(half_min_values)

# Replace negative values with half-min values
coating_subtracted_halfmin = coating_subtracted[metabolite_cols].copy()
for col in coating_subtracted_halfmin.columns:
    coating_subtracted_halfmin.loc[coating_subtracted_halfmin[col] < 0, col] = half_min_values[col]

# These columns only have negative values, so they will be all NaN after half-min imputation
coating_subtracted_halfmin = coating_subtracted_halfmin.dropna(axis=1, how='all')
print(coating_subtracted_halfmin)

Adenosine_OR_Deoxyguanosine     0.428536
alpha_Ketoglutarate             0.030449
AMP_OR_dGMP                     0.124350
Argininosuccinate               0.443107
beta_NAD                        3.456506
                                 ...    
X3_Methyl_2_Oxopentoate        38.770270
X4_Methyl_2_Oxopentoate        53.782202
X5_Hydroxy_Trp                  0.000036
Xanthine                        0.000281
Xylulose_5P                     5.400563
Length: 74, dtype: float64
            Adenosine_OR_Deoxyguanosine  alpha_Ketoglutarate  AMP_OR_dGMP  \
FileNumber                                                                  
115                            1.190899             0.478635     0.623286   
116                            1.074131             0.455061     0.443234   
117                            1.109556             0.456886     0.410719   
118                            0.911291             0.145682     0.334252   
119                            0.875485             0.060898

In [158]:
coating_subtracted

,DateProcessed,TotalRNA,Trial,Lab,CellLine,SampleType,Oxygen,Replicate,Adenosine_OR_Deoxyguanosine,alpha_Ketoglutarate,...,UMP,Uracil,Uridine,X2_Aminoadipic_Acid,X2HG,X3_Methyl_2_Oxopentoate,X4_Methyl_2_Oxopentoate,X5_Hydroxy_Trp,Xanthine,Xylulose_5P
FileNumber,,,,,,,,,,,,,,,,,,,,,
115,X20160502,10.966134,1,Dirks,G566,Cells,Normoxia,1.0,1.190899,0.478635,...,-0.024105,0.007403,0.066769,55.804708,0.942354,365.317548,461.770913,0.001135,0.003899,27.764032
116,X20160502,12.726160,1,Dirks,G566,Cells,Normoxia,2.0,1.074131,0.455061,...,-0.171480,-0.008612,0.070098,63.936240,0.912360,272.852141,438.933896,0.000792,0.000562,29.107027
117,X20160502,14.149547,1,Dirks,G566,Cells,Normoxia,3.0,1.109556,0.456886,...,-0.209223,0.000055,0.044614,37.177856,0.816987,382.278624,427.408551,0.000785,-0.003442,26.701113
118,X20160502,19.906782,1,Dirks,G566,Cells,Normoxia,4.0,0.911291,0.145682,...,-0.352411,0.015298,0.063456,27.878416,0.438273,176.276513,226.560634,0.000415,-0.004333,18.073239
119,X20160502,38.477532,1,Dirks,G566,Cells,Normoxia,5.0,0.875485,0.060898,...,-0.434543,0.003151,0.048378,17.341304,0.231737,77.540541,107.564403,0.000072,-0.009525,10.801126
120,X20160502,20.165580,1,Dirks,G566,Cells,Normoxia,6.0,0.857071,0.305153,...,-0.352992,0.185006,0.118897,31.219403,0.470515,185.016295,212.226478,0.000571,0.036432,17.355569


In [160]:
# Normalize coating_subtracted metabolite columns by TotalRNA for each row
coating_subtracted_normalized = coating_subtracted_halfmin.div(coating_subtracted['TotalRNA'], axis=0)
print(coating_subtracted_normalized)

            Adenosine_OR_Deoxyguanosine  alpha_Ketoglutarate  AMP_OR_dGMP  \
FileNumber                                                                  
115                            0.108598             0.043647     0.056837   
116                            0.084403             0.035758     0.034829   
117                            0.078416             0.032290     0.029027   
118                            0.045778             0.007318     0.016791   
119                            0.022753             0.001583     0.006464   
120                            0.042502             0.015132     0.014181   

            Argininosuccinate  beta_NAD   Citrate  Cytidine  Dissacharide  \
FileNumber                                                                  
115                  0.238536  3.968727  0.202200  0.002471      0.002526   
116                  0.159353  3.037387  0.155651  0.001559      0.001143   
117                  0.144418  2.456719  0.120503  0.001280      0.001021  

In [161]:
median_values = coating_subtracted_normalized.median()
print(median_values)


Adenosine_OR_Deoxyguanosine                       0.062097
alpha_Ketoglutarate                               0.023711
AMP_OR_dGMP                                       0.022909
Argininosuccinate                                 0.110454
beta_NAD                                          1.722378
Citrate                                           0.085034
Cytidine                                          0.000965
Dissacharide                                      0.000600
FAD                                               0.003906
Fumarate                                          0.262937
Glycine                                           0.124093
Guanine                                           0.000381
Guanosine                                         0.041404
Hexose                                            2.288321
Hexose_Phosphate                                  0.539498
Inosine                                           0.037844
Keto_Isovaleric_Acid                              3.0260

In [169]:
median_rows = []
median_rows.append(median_values)
print(median_rows)

[Adenosine_OR_Deoxyguanosine                       0.062097
alpha_Ketoglutarate                               0.023711
AMP_OR_dGMP                                       0.022909
Argininosuccinate                                 0.110454
beta_NAD                                          1.722378
Citrate                                           0.085034
Cytidine                                          0.000965
Dissacharide                                      0.000600
FAD                                               0.003906
Fumarate                                          0.262937
Glycine                                           0.124093
Guanine                                           0.000381
Guanosine                                         0.041404
Hexose                                            2.288321
Hexose_Phosphate                                  0.539498
Inosine                                           0.037844
Keto_Isovaleric_Acid                              3.026

In [170]:
pd.DataFrame(median_rows)

,Adenosine_OR_Deoxyguanosine,alpha_Ketoglutarate,AMP_OR_dGMP,Argininosuccinate,beta_NAD,Citrate,Cytidine,Dissacharide,FAD,Fumarate,...,UDP_Gal_OR_Glc,Uracil,Uridine,X2_Aminoadipic_Acid,X2HG,X3_Methyl_2_Oxopentoate,X4_Methyl_2_Oxopentoate,X5_Hydroxy_Trp,Xanthine,Xylulose_5P
0,0.062097,0.023711,0.022909,0.110454,1.722378,0.085034,0.000965,0.0006,0.003906,0.262937,...,0.180816,0.000378,0.004348,2.087824,0.040536,15.307557,20.793798,0.000042,0.000032,1.397479
